In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Disable tensorflow debugging logs
import tensorflow as tf
import keras_nlp
import tensorflow_text as tf_text
from model import GPT
from utils import *
from config import config

- Download weights

In [2]:
model_name = 'openwt_512_d_512' 
ckpt_dir = f'{model_name}/best-ckpt' 
loader = Loader()
loader.download(ckpt_dir)

In [3]:
conf = Config(config, model_name)

Config openwt_512_d_512/openwt_512_d_512_config.json loaded
{'batch_size': 16, 'buffer_size': 40000, 'shuffle_seed': 32, 'vocab_file': 'wiki_en_vocab', 'min_seq_len': False, 'ckpt_interval': 2000, 'val_steps': 1000, 'train_size': 95, 'vocab_size': 50257, 'seq_len': 512, 'learning_rate': 0.001, 'beta_1': 0.9, 'beta_2': 0.95, 'decay_lr': False, 'decay_steps': 400000, 'alpha': 0.1, 'emb_dim': 512, 'heads': 8, 'mlp_dim': 512, 'depth': 10, 'dropout': 0.0, 'initializer': 'glorot_uniform', 'embedding_initializer': 'glorot_uniform', 'eps': 1e-06, 'mlp_activation': 'gelu'}


In [4]:
model = GPT(vocab_size=conf.vocab_size, 
            seq_len=conf.seq_len, emb_dim=conf.emb_dim,
            heads=conf.heads, mlp_dim=conf.mlp_dim,
            depth=conf.depth, rate=conf.dropout, 
            initializer=conf.initializer)

tokenizer = keras_nlp.models.GPT2Tokenizer.from_preset("gpt2_base_en", 
                sequence_length=conf.seq_len)

- Initialize the model with a tokenized input

In [5]:
context = 'The silver wolf is'
t_context = tokenizer(tf_text.normalize_utf8(context, 'NFKD'))[tf.newaxis, :]

In [6]:
model(t_context)

<tf.Tensor: shape=(1, 512, 50257), dtype=float32, numpy=
array([[[-0.02339119,  0.22609884, -0.18076095, ..., -0.03291423,
          0.00330515,  0.1795414 ],
        [ 0.12021375,  0.22694032, -0.13449436, ..., -0.03326903,
         -0.13123617,  0.20760822],
        [ 0.13187099,  0.12585206, -0.17449507, ..., -0.01563447,
         -0.03126162,  0.16447967],
        ...,
        [ 0.09318852,  0.1439275 , -0.28010085, ..., -0.01569933,
         -0.02199743,  0.18282011],
        [ 0.11052929,  0.11519586, -0.2027312 , ..., -0.09276374,
         -0.02099225,  0.19313608],
        [ 0.10977869,  0.1406471 , -0.15333435, ..., -0.08454964,
         -0.03146493,  0.20974655]]], dtype=float32)>

In [7]:
model.summary()

Model: "gpt"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 token_embedding (TokenEmbe  multiple                  25993728  
 dding)                                                          
                                                                 
 dropout_1 (Dropout)         multiple                  0         
                                                                 
 transformer_block (Transfo  multiple                  1577984   
 rmerBlock)                                                      
                                                                 
 transformer_block_1 (Trans  multiple                  1577984   
 formerBlock)                                                    
                                                                 
 transformer_block_2 (Trans  multiple                  1577984   
 formerBlock)                                                  

- Restore weights

In [8]:
model.restore(ckpt_dir)

Checkpoint restored from openwt_512_d_512/best-ckpt/ckpt-1760000 at step 1760000


In [9]:
text = sample(model, 'The dark ring', max_len=32, k=50, seed=123, temperature=1.0)
print(text)

The dark ring’s more than one-notch ring, ‘Rights to the Edge of the Universe,’ is still in the spotlight


In [10]:
text = sample(model, 'The dark ring', max_len=128, k=50, seed=123, temperature=0.9)
print(text)

The dark ring’s more than one-notch ring, ‘Rights of the End’, the idea of a unique, dynamic ring, with a thin ring that includes a few different types of gear, and a couple of different types of gear over the course of a single year. While many models of the ring use the same high quality and high-quality format, there are a lot of options for those of us who want to add some extra value to the ring.

As you can see in this article:

It’s been done since April 2012 (with the latest results of the


In [11]:
text = sample(model, 'The dark ring', max_len=128, k=50, seed=123, temperature=0.5)
print(text)

The dark ring of the United States’s most famous war machine, the Black Sea, is the first of its kind in the world. The White Sea is home to the world’s most famous war machine, the Black Sea.

The Black Sea is a small town in the northeast corner of the city of San Diego. It is also the largest city in the world.

The Black Sea is a city on the border between the United States and Russia. It is a town in the southernmost part of the U.S., and it is the largest city in the world.

The Black Sea is
